In [1]:
import pymongo
import pandas as pd
import itertools
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus, Word2Vec 
from gensim.models.phrases import Phrases, Phraser, ENGLISH_CONNECTOR_WORDS

In [2]:
sentences = Text8Corpus(datapath('C:/Users/Akkoc/Desktop/Bigram-trigram/wordFile.txt'),
                        max_sentence_length=500000)

In [3]:
first_sentence = next(iter(sentences))

In [4]:
print(first_sentence[:10])

['see', 'referencesthe', 'drone', 'built', 'serve', 'several', 'civilian', 'military', 'usesn', 'type']


In [5]:
len(first_sentence)

264837

In [6]:
phrase_model = Phrases(sentences, min_count=1, threshold=1, connector_words=ENGLISH_CONNECTOR_WORDS)

In [28]:
phrase_list = phrase_model[first_sentence]
print(phrase_list[:10])

['see_referencesthe', 'drone_built', 'serve', 'several', 'civilian_military', 'usesn', 'type', 'hale_high', 'altitude_long', 'endurance']


In [8]:
word2vec_format = [[item] for item in phrase_list]
print(word2vec_format[:10])

[['see_referencesthe'], ['drone_built'], ['serve'], ['several'], ['civilian_military'], ['usesn'], ['type'], ['hale_high'], ['altitude_long'], ['endurance']]


In [9]:
model = Word2Vec(word2vec_format,
                 min_count=1, 
                 workers=8,
                 window=10)

In [13]:
model.wv.most_similar("drones")

[('war_independence', 0.3899497091770172),
 ('bce', 0.38270798325538635),
 ('–_unmanned', 0.37549516558647156),
 ('rests', 0.37393835186958313),
 ('grumman_lockheed', 0.3683522939682007),
 ('maximum_weight', 0.3645557165145874),
 ('postdemonstration', 0.3623088300228119),
 ('feet_wide', 0.36031827330589294),
 ('91', 0.3536728322505951),
 ('per_cent', 0.352961927652359)]

In [14]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase4"]
mycoll = mydb["bigram_wv_coll"]

In [15]:
list_wv = model.wv.index_to_key

In [16]:
for item in list_wv:
    similar_w = model.wv.most_similar(item)
    for word in similar_w:
        x = {"word": item, "similar_word": word[0]}
        mycoll.insert_one(x)

# Trigram

In [65]:
from nltk import ngrams

In [88]:
with open("wordFile.txt",'r', encoding='utf-8') as f:
    trigrams = list(ngrams(f.read().split(),3))

In [89]:
print(trigram[:5])

[('see', 'referencesthe', 'drone'), ('referencesthe', 'drone', 'built'), ('drone', 'built', 'serve'), ('built', 'serve', 'several'), ('serve', 'several', 'civilian')]


In [90]:
trigrm_l = []
for i in range(len(trigram)-1):
    trigrm_l.append(trigram[i][0] + ' ' + trigram[i][1] + ' ' + trigram[i][2] )

In [92]:
print(trigrm_l[:3])
len(trigrm_l)

['see referencesthe drone', 'referencesthe drone built', 'drone built serve']


264834

In [93]:
tgram_list = trigrm_l + first_sentence

In [94]:
len(tgram_list)

529671

In [96]:
word2vec_format_tgram = [[item] for item in tgram_list]
print(word2vec_format[:10])

[['see referencesthe drone'], ['referencesthe drone built'], ['drone built serve'], ['built serve several'], ['serve several civilian'], ['several civilian military'], ['civilian military usesn'], ['military usesn type'], ['usesn type hale'], ['type hale high']]


In [104]:
t_model = Word2Vec(word2vec_format_tgram,
                 min_count=1, 
                 workers=8,
                 window=10)

In [105]:
t_model.wv.most_similar('people')

[('light infrared camera', 0.42211416363716125),
 ('continued vehicles american', 0.4218779504299164),
 ('role cyprusbased squadrons', 0.41908279061317444),
 ('index 2020 important', 0.409993052482605),
 ('interfere enemy communications', 0.40722009539604187),
 ('one control aircraft', 0.40719741582870483),
 ('27 february 2019', 0.3941020965576172),
 ('modified improved upon', 0.39153432846069336),
 ('mars one day', 0.383407860994339),
 ('hours afghanistan dropped', 0.38310322165489197)]

In [106]:
list_tgram_wv = t_model.wv.index_to_key

In [107]:
mycoll2 = mydb["tgram_wv_coll"]

In [ ]:
for item in list_tgram_wv:
    similar_w = t_model.wv.most_similar(item)
    for word in similar_w:
        x = {"word": item, "similar_word": word[0]}
        mycoll2.insert_one(x)